In [ ]:
'''
Initial code was copied from:
https://github.com/jason71995/Keras-GAN-Library
'''

In [1]:
# Loading MHAD data for action1, all persons and all repeatations of each person
from utils.data_loader import data_loader
data_object= data_loader(matlab_action_path='../gan/')
myData, mymin, mymax = data_object.actions_normalised([1], twoD_true_or_threeD_false=False)
myData.shape

(117562, 105)

In [2]:
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D,GlobalAveragePooling2D,LeakyReLU,Conv2DTranspose,Activation,BatchNormalization
from keras.optimizers import Adam

from keras.layers import Dense
from keras import initializers
from keras.layers.core import Dropout



noise_dim = 3

def build_generator(input_shape):

    generator = Sequential()
    
    generator.add(Dense(256,
                        input_dim = noise_dim))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    
#     generator.add(Dense(256))
#     generator.add(BatchNormalization())
#     generator.add(LeakyReLU(0.2))
    

    


    
    generator.add(Dense(105, activation='tanh'))
    return generator


def build_discriminator(input_shape):

    discriminator = Sequential()
    discriminator.add(Dense(256,
                            input_dim=105))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    
#     discriminator.add(Dense(256))
#     discriminator.add(LeakyReLU(0.2))
#     discriminator.add(Dropout(0.3))
    

        

    
    discriminator.add(Dense(1, activation='sigmoid'))

    return discriminator

def build_functions(batch_size, noise_size, image_size, generator, discriminator):

    noise = K.random_normal((batch_size,) + noise_size,0.0,1.0,"float32")
    real_image = K.placeholder((batch_size,) + image_size)

    fake_image = generator(noise)

    d_input = K.concatenate([real_image, fake_image], axis=0)
    pred_real, pred_fake = tf.split(discriminator(d_input), num_or_size_splits = 2, axis = 0)

    pred_real = K.clip(pred_real,K.epsilon(),1-K.epsilon())
    pred_fake = K.clip(pred_fake,K.epsilon(),1-K.epsilon())

    d_loss = -(K.mean(K.log(pred_real)) + K.mean(K.log(1-pred_fake)))
    g_loss = -K.mean(K.log(pred_fake))

    # get updates of mean and variance in batch normalization layers
    d_updates = discriminator.get_updates_for([d_input])
    g_updates = generator.get_updates_for([noise])

    d_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(d_loss, discriminator.trainable_weights)
    d_train = K.function([real_image, K.learning_phase()], [d_loss],d_updates + d_training_updates)

    g_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(g_loss, generator.trainable_weights)
    g_train = K.function([real_image, K.learning_phase()], [g_loss], g_updates + g_training_updates)

    return d_train,g_train

Using TensorFlow backend.


In [4]:
import keras

#from gan_libs.DCGAN import build_generator, build_discriminator, build_functions
# from gan_libs.LSGAN import build_generator, build_discriminator, build_functions
# from gan_libs.SNGAN import build_generator, build_discriminator, build_functions
# from gan_libs.WGAN_GP import build_generator, build_discriminator, build_functions

from utils.common import set_gpu_config, predict_images
from utils.draw_pose import draw_pose
import numpy as np

#set_gpu_config("0",0.5)

epoch = 50 + 1
image_size = (1,1,105)
noise_size = (1,1,3)
batch_size = 16

x_train = myData
np.random.shuffle(x_train)

generator = build_generator(noise_size)
print('gen params:',generator.count_params())
discriminator = build_discriminator(image_size)
print('dis params:',discriminator.count_params())
d_train, g_train = build_functions(batch_size, noise_size, image_size, generator, discriminator)

# generator.load_weights("e30_generator.h5".format(e))
# discriminator.load_weights("e30_discriminator.h5".format(e))


number_of_all_data = x_train.shape[0]
number_of_batches = int(number_of_all_data/batch_size)
print('Number of Batches passed in each epoch: ',number_of_batches)

import time
start_time = time.time()
is_first_epoch = True

for e in range(epoch):
    index = 0
    for batch in range(number_of_batches):
        real_images = x_train[index:index+batch_size]
        index =+ batch_size
        real_images.shape = (batch_size,1,1,105)
        d_loss, = d_train([real_images, 1])
        g_loss, = g_train([real_images, 1])
        if np.random.randint(low = 0, high = 100) == 1:   
            print ("[{0}/{1}] d_loss: {2:.4}, g_loss: {3:.4}".format(e, epoch, d_loss, g_loss))
            #generating a sample
            image = generator.predict(np.zeros(shape=(1,3)))
            image = np.array(image)
            draw_pose(image.reshape(105),'output',"_ThreeLatents_e{0}_batch{1}".format(e,batch))
    
    if(is_first_epoch):
        elapsed_time = time.time() - start_time 
        print('\n\nTime Taken for single epoch:')
        print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
        is_first_epoch = False
    
    if e % 50 == 0 and e > 0:
        generator.save_weights("_ThreeLatents_e{0}_generator.h5".format(e))
        discriminator.save_weights("_ThreeLatents_e{0}_discriminator.h5".format(e))
 
#just monitoring:
##########################################################################################
elapsed_time = time.time() - start_time 
print('\n\n\n\nNumber of parameter for the Generator and discriminator respectively:\n')
print(generator.count_params())
print('')
print(discriminator.count_params())
print('\n\nNumber of Epochs and steps for each epoch:\n')
print('epochs: ',epoch, '   batches: ', number_of_batches)

print('\n\nTime Taken:')
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
##########################################################################################
#NOTE: previously in each epoch 1000 steps were iterated. in each iteration data was permitated and a batch was chosen. 
#now data is shuffled first and there are number_of_data/batch_size steps to be passed in a single epoch. 
# since 200 epochs and 1000 steps each was used, now 25 epochs and ˜7300 steps is used so the balance is kept.

gen params: 29033
dis params: 27393
Number of Batches passed in each epoch:  7347
[0/51] d_loss: 0.9896, g_loss: 0.6316
[0/51] d_loss: 0.8889, g_loss: 0.7088
[0/51] d_loss: 0.9606, g_loss: 0.7459
[0/51] d_loss: 1.086, g_loss: 0.6169
[0/51] d_loss: 1.233, g_loss: 0.8415
[0/51] d_loss: 1.229, g_loss: 0.8513
[0/51] d_loss: 1.36, g_loss: 0.8986
[0/51] d_loss: 1.317, g_loss: 0.7908
[0/51] d_loss: 1.303, g_loss: 0.9429
[0/51] d_loss: 1.329, g_loss: 0.9148
[0/51] d_loss: 1.327, g_loss: 0.9209
[0/51] d_loss: 1.296, g_loss: 0.8635
[0/51] d_loss: 1.303, g_loss: 0.8957
[0/51] d_loss: 1.21, g_loss: 0.8668
[0/51] d_loss: 1.256, g_loss: 0.7465
[0/51] d_loss: 1.306, g_loss: 0.7407
[0/51] d_loss: 1.308, g_loss: 0.7199
[0/51] d_loss: 1.336, g_loss: 0.7467
[0/51] d_loss: 1.248, g_loss: 0.6717
[0/51] d_loss: 1.309, g_loss: 0.7615
[0/51] d_loss: 1.267, g_loss: 0.7253
[0/51] d_loss: 1.289, g_loss: 0.7334
[0/51] d_loss: 1.262, g_loss: 0.7289
[0/51] d_loss: 1.256, g_loss: 0.7185
[0/51] d_loss: 1.296, g_loss:

[5/51] d_loss: 1.167, g_loss: 0.9966
[5/51] d_loss: 1.163, g_loss: 0.7861
[5/51] d_loss: 1.076, g_loss: 1.083
[5/51] d_loss: 1.11, g_loss: 0.9651
[5/51] d_loss: 1.114, g_loss: 0.7824
[5/51] d_loss: 1.133, g_loss: 0.849
[5/51] d_loss: 1.139, g_loss: 0.8311
[5/51] d_loss: 1.062, g_loss: 0.9708
[5/51] d_loss: 1.116, g_loss: 1.052
[5/51] d_loss: 1.087, g_loss: 0.891
[5/51] d_loss: 1.101, g_loss: 0.8799
[5/51] d_loss: 1.088, g_loss: 0.8422
[5/51] d_loss: 1.101, g_loss: 1.081
[5/51] d_loss: 1.123, g_loss: 0.7804
[5/51] d_loss: 1.09, g_loss: 1.006
[5/51] d_loss: 1.115, g_loss: 0.7982
[5/51] d_loss: 1.067, g_loss: 0.9833
[5/51] d_loss: 1.107, g_loss: 1.014
[5/51] d_loss: 1.052, g_loss: 0.9582
[5/51] d_loss: 1.147, g_loss: 1.022
[5/51] d_loss: 1.124, g_loss: 0.9299
[5/51] d_loss: 1.156, g_loss: 1.045
[5/51] d_loss: 1.118, g_loss: 1.092
[5/51] d_loss: 1.163, g_loss: 1.065
[5/51] d_loss: 1.12, g_loss: 0.9379
[5/51] d_loss: 1.107, g_loss: 0.9492
[5/51] d_loss: 1.274, g_loss: 0.9573
[5/51] d_loss: 

[8/51] d_loss: 0.983, g_loss: 1.306
[8/51] d_loss: 1.062, g_loss: 1.22
[8/51] d_loss: 0.9343, g_loss: 1.049
[8/51] d_loss: 1.009, g_loss: 0.8934
[8/51] d_loss: 1.057, g_loss: 1.245
[8/51] d_loss: 1.133, g_loss: 1.027
[8/51] d_loss: 1.026, g_loss: 0.7512
[8/51] d_loss: 0.8708, g_loss: 1.181
[8/51] d_loss: 0.9456, g_loss: 1.245
[8/51] d_loss: 1.003, g_loss: 1.337
[8/51] d_loss: 0.9517, g_loss: 1.378
[8/51] d_loss: 0.8945, g_loss: 1.237
[8/51] d_loss: 1.041, g_loss: 1.491
[8/51] d_loss: 0.9137, g_loss: 0.8386
[8/51] d_loss: 0.9543, g_loss: 0.9604
[8/51] d_loss: 1.056, g_loss: 1.271
[8/51] d_loss: 1.071, g_loss: 1.415
[8/51] d_loss: 1.121, g_loss: 1.122
[8/51] d_loss: 1.002, g_loss: 1.258
[8/51] d_loss: 1.045, g_loss: 1.441
[8/51] d_loss: 0.9513, g_loss: 1.105
[8/51] d_loss: 1.034, g_loss: 1.108
[8/51] d_loss: 1.046, g_loss: 1.305
[8/51] d_loss: 0.9795, g_loss: 1.193
[8/51] d_loss: 0.8886, g_loss: 1.359
[8/51] d_loss: 0.9838, g_loss: 1.198
[8/51] d_loss: 0.9656, g_loss: 1.131
[8/51] d_loss

[11/51] d_loss: 0.8062, g_loss: 1.366
[11/51] d_loss: 0.8275, g_loss: 1.795
[11/51] d_loss: 0.856, g_loss: 1.721
[11/51] d_loss: 0.7545, g_loss: 1.579
[11/51] d_loss: 0.8458, g_loss: 1.474
[11/51] d_loss: 0.9783, g_loss: 1.897
[11/51] d_loss: 0.9859, g_loss: 1.739
[11/51] d_loss: 0.8723, g_loss: 1.476
[11/51] d_loss: 0.8254, g_loss: 0.9054
[11/51] d_loss: 0.892, g_loss: 1.498
[11/51] d_loss: 0.9451, g_loss: 1.135
[11/51] d_loss: 1.138, g_loss: 1.246
[11/51] d_loss: 0.7052, g_loss: 1.201
[11/51] d_loss: 0.8936, g_loss: 1.4
[11/51] d_loss: 0.8741, g_loss: 1.991
[11/51] d_loss: 0.7418, g_loss: 1.491
[11/51] d_loss: 0.8465, g_loss: 1.895
[11/51] d_loss: 0.9204, g_loss: 1.453
[11/51] d_loss: 0.8306, g_loss: 1.167
[11/51] d_loss: 0.7257, g_loss: 1.612
[11/51] d_loss: 0.8264, g_loss: 1.641
[11/51] d_loss: 0.71, g_loss: 1.348
[11/51] d_loss: 0.6484, g_loss: 1.944
[11/51] d_loss: 0.7167, g_loss: 2.03
[11/51] d_loss: 0.8229, g_loss: 1.105
[11/51] d_loss: 0.8676, g_loss: 1.965
[11/51] d_loss: 0.9

[14/51] d_loss: 1.07, g_loss: 1.534
[14/51] d_loss: 0.7282, g_loss: 1.366
[14/51] d_loss: 0.5992, g_loss: 3.05
[14/51] d_loss: 0.5542, g_loss: 1.686
[14/51] d_loss: 1.084, g_loss: 0.7637
[14/51] d_loss: 0.7061, g_loss: 1.291
[14/51] d_loss: 0.7243, g_loss: 1.338
[14/51] d_loss: 0.7466, g_loss: 1.386
[14/51] d_loss: 0.9044, g_loss: 2.555
[14/51] d_loss: 0.7439, g_loss: 1.421
[14/51] d_loss: 0.7069, g_loss: 1.578
[14/51] d_loss: 0.779, g_loss: 1.79
[14/51] d_loss: 0.6813, g_loss: 1.0
[14/51] d_loss: 0.7234, g_loss: 2.205
[14/51] d_loss: 0.7024, g_loss: 1.682
[14/51] d_loss: 0.6744, g_loss: 1.9
[14/51] d_loss: 0.7777, g_loss: 1.596
[14/51] d_loss: 0.6972, g_loss: 1.502
[14/51] d_loss: 0.7014, g_loss: 1.991
[14/51] d_loss: 0.6087, g_loss: 2.018
[14/51] d_loss: 0.6798, g_loss: 1.95
[14/51] d_loss: 0.83, g_loss: 1.425
[14/51] d_loss: 0.5583, g_loss: 1.751
[14/51] d_loss: 0.6039, g_loss: 2.014
[14/51] d_loss: 0.7632, g_loss: 1.752
[14/51] d_loss: 0.7236, g_loss: 2.245
[14/51] d_loss: 0.787, g

...

[38/51] d_loss: 0.6636, g_loss: 1.752
[38/51] d_loss: 0.7956, g_loss: 1.05
[38/51] d_loss: 0.6864, g_loss: 2.608
[38/51] d_loss: 0.5613, g_loss: 2.943
[38/51] d_loss: 0.5602, g_loss: 2.883
[38/51] d_loss: 0.5459, g_loss: 3.19
[38/51] d_loss: 0.5235, g_loss: 2.633
[38/51] d_loss: 0.8439, g_loss: 2.054
[38/51] d_loss: 0.6888, g_loss: 1.846
[38/51] d_loss: 1.072, g_loss: 0.7104
[38/51] d_loss: 1.14, g_loss: 2.116
[38/51] d_loss: 1.319, g_loss: 2.488
[38/51] d_loss: 0.5102, g_loss: 2.736
[38/51] d_loss: 0.9543, g_loss: 1.059
[38/51] d_loss: 0.98, g_loss: 1.597
[38/51] d_loss: 0.6954, g_loss: 2.989
[38/51] d_loss: 0.5053, g_loss: 2.953
[38/51] d_loss: 0.7443, g_loss: 2.841
[38/51] d_loss: 0.5142, g_loss: 4.133
[38/51] d_loss: 0.7272, g_loss: 1.478
[39/51] d_loss: 1.001, g_loss: 1.932
[39/51] d_loss: 0.5164, g_loss: 2.546
[39/51] d_loss: 1.003, g_loss: 2.42
[39/51] d_loss: 0.7995, g_loss: 1.075
[39/51] d_loss: 0.3727, g_loss: 3.584
[39/51] d_loss: 0.5819, g_loss: 2.947
[39/51] d_loss: 0.5059

[42/51] d_loss: 0.981, g_loss: 2.807
[42/51] d_loss: 0.5309, g_loss: 2.011
[42/51] d_loss: 0.5427, g_loss: 2.029
[42/51] d_loss: 0.7596, g_loss: 1.588
[42/51] d_loss: 0.8614, g_loss: 2.897
[42/51] d_loss: 0.5267, g_loss: 2.195
[42/51] d_loss: 1.806, g_loss: 1.328
[42/51] d_loss: 0.3809, g_loss: 3.606
[42/51] d_loss: 0.6143, g_loss: 2.364
[42/51] d_loss: 0.6429, g_loss: 2.591
[42/51] d_loss: 0.5324, g_loss: 2.489
[42/51] d_loss: 0.9993, g_loss: 3.009
[42/51] d_loss: 0.754, g_loss: 1.585
[42/51] d_loss: 0.9278, g_loss: 2.264
[42/51] d_loss: 0.7734, g_loss: 1.604
[42/51] d_loss: 0.5959, g_loss: 1.5
[42/51] d_loss: 0.9217, g_loss: 2.837
[42/51] d_loss: 1.511, g_loss: 1.527
[42/51] d_loss: 0.8414, g_loss: 2.028
[42/51] d_loss: 0.6503, g_loss: 2.103
[42/51] d_loss: 0.6482, g_loss: 2.414
[42/51] d_loss: 1.275, g_loss: 0.3922
[42/51] d_loss: 0.6103, g_loss: 2.295
[42/51] d_loss: 0.5327, g_loss: 3.017
[42/51] d_loss: 0.756, g_loss: 2.807
[42/51] d_loss: 0.9378, g_loss: 1.84
[42/51] d_loss: 0.61

[45/51] d_loss: 0.9332, g_loss: 2.958
[45/51] d_loss: 0.741, g_loss: 1.579
[45/51] d_loss: 0.85, g_loss: 2.236
[45/51] d_loss: 0.6606, g_loss: 3.706
[45/51] d_loss: 1.137, g_loss: 1.133
[45/51] d_loss: 0.5924, g_loss: 1.725
[45/51] d_loss: 0.6812, g_loss: 1.624
[45/51] d_loss: 0.7784, g_loss: 2.287
[45/51] d_loss: 0.7565, g_loss: 2.023
[45/51] d_loss: 0.8955, g_loss: 2.264
[45/51] d_loss: 0.633, g_loss: 2.455
[45/51] d_loss: 0.6884, g_loss: 1.598
[45/51] d_loss: 0.5637, g_loss: 1.899
[45/51] d_loss: 1.435, g_loss: 1.69
[45/51] d_loss: 0.8386, g_loss: 2.778
[45/51] d_loss: 0.6526, g_loss: 1.663
[45/51] d_loss: 0.6278, g_loss: 1.168
[45/51] d_loss: 0.5925, g_loss: 1.583
[45/51] d_loss: 1.08, g_loss: 3.424
[45/51] d_loss: 0.8218, g_loss: 3.055
[45/51] d_loss: 0.6684, g_loss: 2.601
[45/51] d_loss: 1.231, g_loss: 1.435
[45/51] d_loss: 0.939, g_loss: 1.016
[45/51] d_loss: 0.9302, g_loss: 2.826
[45/51] d_loss: 1.073, g_loss: 2.4
[45/51] d_loss: 0.4997, g_loss: 3.624
[45/51] d_loss: 0.7196, g_

[48/51] d_loss: 0.7043, g_loss: 2.049
[48/51] d_loss: 0.6501, g_loss: 2.293
[48/51] d_loss: 1.252, g_loss: 1.891
[48/51] d_loss: 1.362, g_loss: 1.985
[48/51] d_loss: 0.9952, g_loss: 2.368
[48/51] d_loss: 0.6267, g_loss: 2.896
[48/51] d_loss: 1.225, g_loss: 2.57
[48/51] d_loss: 0.7626, g_loss: 0.9446
[48/51] d_loss: 1.065, g_loss: 2.347
[48/51] d_loss: 1.077, g_loss: 3.016
[48/51] d_loss: 1.58, g_loss: 2.039
[48/51] d_loss: 1.745, g_loss: 2.156
[48/51] d_loss: 1.239, g_loss: 1.2
[48/51] d_loss: 1.008, g_loss: 1.42
[48/51] d_loss: 0.8569, g_loss: 2.29
[48/51] d_loss: 0.7482, g_loss: 2.227
[48/51] d_loss: 0.7235, g_loss: 1.495
[48/51] d_loss: 1.297, g_loss: 1.733
[48/51] d_loss: 1.051, g_loss: 1.59
[48/51] d_loss: 0.9056, g_loss: 0.8635
[48/51] d_loss: 0.6664, g_loss: 2.596
[48/51] d_loss: 0.8283, g_loss: 0.8188
[48/51] d_loss: 0.7321, g_loss: 2.181
[48/51] d_loss: 1.663, g_loss: 1.776
[48/51] d_loss: 0.8992, g_loss: 1.431
[48/51] d_loss: 0.7398, g_loss: 0.8211
[48/51] d_loss: 1.147, g_lo

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D,GlobalAveragePooling2D,LeakyReLU,Conv2DTranspose,Activation,BatchNormalization
from keras.optimizers import Adam

from keras.layers import Dense
from keras import initializers
from keras.layers.core import Dropout



noise_dim = 2

def build_generator(input_shape):

    generator = Sequential()
    
    generator.add(Dense(256,
                        input_dim = noise_dim))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    
#     generator.add(Dense(256))
#     generator.add(BatchNormalization())
#     generator.add(LeakyReLU(0.2))
    

    


    
    generator.add(Dense(105, activation='tanh'))
    return generator


def build_discriminator(input_shape):

    discriminator = Sequential()
    discriminator.add(Dense(256,
                            input_dim=105))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    
#     discriminator.add(Dense(256))
#     discriminator.add(LeakyReLU(0.2))
#     discriminator.add(Dropout(0.3))
    

        

    
    discriminator.add(Dense(1, activation='sigmoid'))

    return discriminator

def build_functions(batch_size, noise_size, image_size, generator, discriminator):

    noise = K.random_normal((batch_size,) + noise_size,0.0,1.0,"float32")
    real_image = K.placeholder((batch_size,) + image_size)

    fake_image = generator(noise)

    d_input = K.concatenate([real_image, fake_image], axis=0)
    pred_real, pred_fake = tf.split(discriminator(d_input), num_or_size_splits = 2, axis = 0)

    pred_real = K.clip(pred_real,K.epsilon(),1-K.epsilon())
    pred_fake = K.clip(pred_fake,K.epsilon(),1-K.epsilon())

    d_loss = -(K.mean(K.log(pred_real)) + K.mean(K.log(1-pred_fake)))
    g_loss = -K.mean(K.log(pred_fake))

    # get updates of mean and variance in batch normalization layers
    d_updates = discriminator.get_updates_for([d_input])
    g_updates = generator.get_updates_for([noise])

    d_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(d_loss, discriminator.trainable_weights)
    d_train = K.function([real_image, K.learning_phase()], [d_loss],d_updates + d_training_updates)

    g_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(g_loss, generator.trainable_weights)
    g_train = K.function([real_image, K.learning_phase()], [g_loss], g_updates + g_training_updates)

    return d_train,g_train

In [ ]:
import keras

#from gan_libs.DCGAN import build_generator, build_discriminator, build_functions
# from gan_libs.LSGAN import build_generator, build_discriminator, build_functions
# from gan_libs.SNGAN import build_generator, build_discriminator, build_functions
# from gan_libs.WGAN_GP import build_generator, build_discriminator, build_functions

from utils.common import set_gpu_config, predict_images
from utils.draw_pose import draw_pose
import numpy as np

#set_gpu_config("0",0.5)

epoch = 50 + 1
image_size = (1,1,105)
noise_size = (1,1,2)
batch_size = 16

x_train = myData
np.random.shuffle(x_train)

generator = build_generator(noise_size)
print('gen params:',generator.count_params())
discriminator = build_discriminator(image_size)
print('dis params:',discriminator.count_params())
d_train, g_train = build_functions(batch_size, noise_size, image_size, generator, discriminator)

# generator.load_weights("e30_generator.h5".format(e))
# discriminator.load_weights("e30_discriminator.h5".format(e))


number_of_all_data = x_train.shape[0]
number_of_batches = int(number_of_all_data/batch_size)
print('Number of Batches passed in each epoch: ',number_of_batches)

import time
start_time = time.time()
is_first_epoch = True

for e in range(epoch):
    index = 0
    for batch in range(number_of_batches):
        real_images = x_train[index:index+batch_size]
        index =+ batch_size
        real_images.shape = (batch_size,1,1,105)
        d_loss, = d_train([real_images, 1])
        g_loss, = g_train([real_images, 1])
        if np.random.randint(low = 0, high = 100) == 1:   
            print ("[{0}/{1}] d_loss: {2:.4}, g_loss: {3:.4}".format(e, epoch, d_loss, g_loss))
            #generating a sample
            image = generator.predict(np.zeros(shape=(1,2)))
            image = np.array(image)
            draw_pose(image.reshape(105),'output',"_TwoLatents_e{0}_batch{1}".format(e,batch))
    
    if(is_first_epoch):
        elapsed_time = time.time() - start_time 
        print('\n\nTime Taken for single epoch:')
        print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
        is_first_epoch = False
    
    if e % 50 == 0 and e > 0:
        generator.save_weights("_TwoLatents_e{0}_generator.h5".format(e))
        discriminator.save_weights("_TwoLatents_e{0}_discriminator.h5".format(e))
 
#just monitoring:
##########################################################################################
elapsed_time = time.time() - start_time 
print('\n\n\n\nNumber of parameter for the Generator and discriminator respectively:\n')
print(generator.count_params())
print('')
print(discriminator.count_params())
print('\n\nNumber of Epochs and steps for each epoch:\n')
print('epochs: ',epoch, '   batches: ', number_of_batches)

print('\n\nTime Taken:')
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
##########################################################################################
#NOTE: previously in each epoch 1000 steps were iterated. in each iteration data was permitated and a batch was chosen. 
#now data is shuffled first and there are number_of_data/batch_size steps to be passed in a single epoch. 
# since 200 epochs and 1000 steps each was used, now 25 epochs and ˜7300 steps is used so the balance is kept.

gen params: 28777
dis params: 27393
Number of Batches passed in each epoch:  7347
[0/51] d_loss: 1.098, g_loss: 0.5659
[0/51] d_loss: 1.041, g_loss: 0.6934
[0/51] d_loss: 0.9398, g_loss: 0.6619
[0/51] d_loss: 0.9874, g_loss: 0.6909
[0/51] d_loss: 1.295, g_loss: 0.7258
[0/51] d_loss: 1.364, g_loss: 0.6481
[0/51] d_loss: 1.306, g_loss: 0.7856
[0/51] d_loss: 1.257, g_loss: 0.7492
[0/51] d_loss: 1.34, g_loss: 0.7855
[0/51] d_loss: 1.34, g_loss: 0.8154
[0/51] d_loss: 1.364, g_loss: 0.8405
[0/51] d_loss: 1.398, g_loss: 0.8404
[0/51] d_loss: 1.404, g_loss: 0.7274
[0/51] d_loss: 1.366, g_loss: 0.7795
[0/51] d_loss: 1.333, g_loss: 0.735
[0/51] d_loss: 1.334, g_loss: 0.7159
[0/51] d_loss: 1.398, g_loss: 0.7097
[0/51] d_loss: 1.362, g_loss: 0.7022
[0/51] d_loss: 1.364, g_loss: 0.7256
[0/51] d_loss: 1.372, g_loss: 0.6812
[0/51] d_loss: 1.36, g_loss: 0.7081
[0/51] d_loss: 1.361, g_loss: 0.6872
[0/51] d_loss: 1.386, g_loss: 0.6784
[0/51] d_loss: 1.336, g_loss: 0.6756
[0/51] d_loss: 1.347, g_loss: 0.

[2/51] d_loss: 1.253, g_loss: 0.7623
[2/51] d_loss: 1.211, g_loss: 0.7805
[2/51] d_loss: 1.179, g_loss: 0.7948
[2/51] d_loss: 1.298, g_loss: 0.7729
[2/51] d_loss: 1.234, g_loss: 0.757
[2/51] d_loss: 1.282, g_loss: 0.7926
[2/51] d_loss: 1.285, g_loss: 0.7922
[2/51] d_loss: 1.213, g_loss: 0.7755
[2/51] d_loss: 1.262, g_loss: 0.8065
[2/51] d_loss: 1.246, g_loss: 0.814
[2/51] d_loss: 1.232, g_loss: 0.7813
[2/51] d_loss: 1.302, g_loss: 0.8584
[2/51] d_loss: 1.256, g_loss: 0.8075
[2/51] d_loss: 1.187, g_loss: 0.745
[2/51] d_loss: 1.261, g_loss: 0.79
[2/51] d_loss: 1.227, g_loss: 0.8037
[2/51] d_loss: 1.249, g_loss: 0.7974
[2/51] d_loss: 1.274, g_loss: 0.8682
[2/51] d_loss: 1.224, g_loss: 0.8598
[2/51] d_loss: 1.244, g_loss: 0.7927
[2/51] d_loss: 1.206, g_loss: 0.8207
[2/51] d_loss: 1.204, g_loss: 0.8335
[2/51] d_loss: 1.214, g_loss: 0.8143
[2/51] d_loss: 1.251, g_loss: 0.786
[2/51] d_loss: 1.228, g_loss: 0.7763
[2/51] d_loss: 1.24, g_loss: 0.7869
[2/51] d_loss: 1.219, g_loss: 0.8115
[2/51] d

[5/51] d_loss: 0.9808, g_loss: 1.076
[5/51] d_loss: 1.109, g_loss: 0.9975
[5/51] d_loss: 0.8956, g_loss: 0.9036
[5/51] d_loss: 1.071, g_loss: 1.04
[5/51] d_loss: 1.122, g_loss: 0.8975
[5/51] d_loss: 1.1, g_loss: 1.064
[5/51] d_loss: 1.128, g_loss: 1.194
[5/51] d_loss: 1.084, g_loss: 1.092
[5/51] d_loss: 1.111, g_loss: 0.9362
[5/51] d_loss: 1.135, g_loss: 1.032
[5/51] d_loss: 0.9609, g_loss: 0.948
[5/51] d_loss: 1.063, g_loss: 0.9461
[5/51] d_loss: 1.149, g_loss: 1.147
[5/51] d_loss: 1.109, g_loss: 0.912
[5/51] d_loss: 0.9524, g_loss: 0.9532
[5/51] d_loss: 0.9125, g_loss: 1.071
[5/51] d_loss: 1.165, g_loss: 1.134
[5/51] d_loss: 1.196, g_loss: 0.8129
[5/51] d_loss: 1.021, g_loss: 0.9224
[5/51] d_loss: 0.9177, g_loss: 0.9512
[5/51] d_loss: 1.098, g_loss: 0.9608
[5/51] d_loss: 1.069, g_loss: 0.901
[5/51] d_loss: 1.009, g_loss: 0.9398
[5/51] d_loss: 1.123, g_loss: 0.9733
[5/51] d_loss: 1.034, g_loss: 1.082
[5/51] d_loss: 1.161, g_loss: 1.042
[5/51] d_loss: 0.9369, g_loss: 0.8792
[5/51] d_lo

[8/51] d_loss: 0.6398, g_loss: 1.416
[8/51] d_loss: 0.8231, g_loss: 1.159
[8/51] d_loss: 0.7216, g_loss: 1.473
[8/51] d_loss: 0.9054, g_loss: 1.199
[8/51] d_loss: 0.9494, g_loss: 0.9337
[8/51] d_loss: 0.713, g_loss: 1.63
[8/51] d_loss: 0.7626, g_loss: 1.66
[8/51] d_loss: 0.9069, g_loss: 1.356
[8/51] d_loss: 0.5397, g_loss: 1.819
[8/51] d_loss: 0.7969, g_loss: 1.586
[8/51] d_loss: 0.6736, g_loss: 1.219
[8/51] d_loss: 0.9555, g_loss: 1.257
[8/51] d_loss: 0.8955, g_loss: 1.787
[8/51] d_loss: 0.9259, g_loss: 0.9577
[8/51] d_loss: 1.103, g_loss: 1.515
[8/51] d_loss: 0.7733, g_loss: 0.8057
[8/51] d_loss: 0.8735, g_loss: 1.431
[8/51] d_loss: 0.8457, g_loss: 1.672
[8/51] d_loss: 0.7739, g_loss: 1.16
[8/51] d_loss: 0.7266, g_loss: 0.9973
[8/51] d_loss: 0.7301, g_loss: 1.568
[8/51] d_loss: 0.797, g_loss: 1.759
[8/51] d_loss: 0.8054, g_loss: 1.35
[8/51] d_loss: 0.8864, g_loss: 1.684
[8/51] d_loss: 0.8861, g_loss: 1.484
[8/51] d_loss: 0.8569, g_loss: 1.888
[8/51] d_loss: 0.8103, g_loss: 1.506
[8/5

[11/51] d_loss: 0.9972, g_loss: 1.381
[11/51] d_loss: 0.7321, g_loss: 1.982
[11/51] d_loss: 0.8142, g_loss: 1.224
[11/51] d_loss: 0.5426, g_loss: 2.541
[11/51] d_loss: 0.7467, g_loss: 1.739
[11/51] d_loss: 0.8499, g_loss: 2.176
[11/51] d_loss: 0.8024, g_loss: 1.36
[11/51] d_loss: 0.4655, g_loss: 3.724
[11/51] d_loss: 0.6831, g_loss: 2.749
[11/51] d_loss: 0.5917, g_loss: 2.02
[11/51] d_loss: 0.7687, g_loss: 1.654
[11/51] d_loss: 0.7639, g_loss: 1.96
[11/51] d_loss: 0.8997, g_loss: 1.457
[11/51] d_loss: 0.622, g_loss: 2.434
[11/51] d_loss: 0.7804, g_loss: 1.391
[12/51] d_loss: 0.6852, g_loss: 3.175
[12/51] d_loss: 0.9751, g_loss: 1.722
[12/51] d_loss: 0.7373, g_loss: 2.378
[12/51] d_loss: 0.5963, g_loss: 2.231
[12/51] d_loss: 0.5278, g_loss: 3.259
[12/51] d_loss: 0.8026, g_loss: 2.146
[12/51] d_loss: 0.8404, g_loss: 1.616
[12/51] d_loss: 0.422, g_loss: 2.608
[12/51] d_loss: 1.141, g_loss: 2.393
[12/51] d_loss: 0.5891, g_loss: 1.992
[12/51] d_loss: 1.03, g_loss: 1.105
[12/51] d_loss: 0.49

...

[32/51] d_loss: 0.6368, g_loss: 3.694
[32/51] d_loss: 1.177, g_loss: 2.275
[32/51] d_loss: 0.6939, g_loss: 3.053
[32/51] d_loss: 2.286, g_loss: 1.442
[32/51] d_loss: 0.4028, g_loss: 2.605
[32/51] d_loss: 0.5512, g_loss: 2.055
[32/51] d_loss: 0.435, g_loss: 2.924
[32/51] d_loss: 0.8211, g_loss: 1.819
[32/51] d_loss: 0.4188, g_loss: 1.645
[32/51] d_loss: 0.8633, g_loss: 1.892
[32/51] d_loss: 0.4383, g_loss: 2.913
[32/51] d_loss: 0.4151, g_loss: 2.144
[32/51] d_loss: 0.8763, g_loss: 2.363
[32/51] d_loss: 0.4555, g_loss: 2.543
[32/51] d_loss: 0.4629, g_loss: 2.575
[32/51] d_loss: 0.4949, g_loss: 3.246
[32/51] d_loss: 0.7328, g_loss: 1.607
[32/51] d_loss: 0.4381, g_loss: 2.653
[32/51] d_loss: 0.4559, g_loss: 2.876
[32/51] d_loss: 0.3973, g_loss: 2.965
[32/51] d_loss: 0.3778, g_loss: 3.977
[32/51] d_loss: 0.5673, g_loss: 1.86
[32/51] d_loss: 0.4121, g_loss: 4.184
[32/51] d_loss: 0.6154, g_loss: 4.321
[32/51] d_loss: 0.2782, g_loss: 3.876
[32/51] d_loss: 0.4629, g_loss: 4.35
[32/51] d_loss: 0

[34/51] d_loss: 0.6599, g_loss: 2.702
[34/51] d_loss: 0.3997, g_loss: 3.454
[34/51] d_loss: 0.5706, g_loss: 0.7946
[34/51] d_loss: 0.7024, g_loss: 2.506
[34/51] d_loss: 0.4602, g_loss: 2.141
[35/51] d_loss: 0.4997, g_loss: 1.618
[35/51] d_loss: 0.7508, g_loss: 2.18
[35/51] d_loss: 0.689, g_loss: 2.895
[35/51] d_loss: 0.773, g_loss: 1.665
[35/51] d_loss: 0.5025, g_loss: 1.621
[35/51] d_loss: 0.5434, g_loss: 2.112
[35/51] d_loss: 1.036, g_loss: 4.118
[35/51] d_loss: 0.515, g_loss: 2.878
[35/51] d_loss: 0.9835, g_loss: 3.007
[35/51] d_loss: 0.6764, g_loss: 1.716
[35/51] d_loss: 0.8788, g_loss: 2.487
[35/51] d_loss: 0.7492, g_loss: 1.996
[35/51] d_loss: 0.5339, g_loss: 3.048
[35/51] d_loss: 0.8705, g_loss: 1.198
[35/51] d_loss: 0.6017, g_loss: 3.5
[35/51] d_loss: 0.3714, g_loss: 1.736
[35/51] d_loss: 0.5152, g_loss: 1.747
[35/51] d_loss: 0.548, g_loss: 1.629
[35/51] d_loss: 0.6435, g_loss: 2.924
[35/51] d_loss: 0.5556, g_loss: 2.476
[35/51] d_loss: 0.6664, g_loss: 1.912
[35/51] d_loss: 0.5

[38/51] d_loss: 0.8789, g_loss: 3.302
[38/51] d_loss: 0.8806, g_loss: 2.551
[38/51] d_loss: 0.6041, g_loss: 2.254
[38/51] d_loss: 0.7441, g_loss: 0.8394
[38/51] d_loss: 0.6121, g_loss: 1.619
[38/51] d_loss: 0.4355, g_loss: 1.839
[38/51] d_loss: 0.7837, g_loss: 2.894
[38/51] d_loss: 0.6948, g_loss: 1.262
[38/51] d_loss: 0.5475, g_loss: 2.05
[38/51] d_loss: 1.444, g_loss: 2.023
[38/51] d_loss: 0.651, g_loss: 2.439
[38/51] d_loss: 0.5177, g_loss: 2.08
[38/51] d_loss: 0.4964, g_loss: 2.393
[38/51] d_loss: 0.7618, g_loss: 1.73
[38/51] d_loss: 0.8256, g_loss: 2.917
[38/51] d_loss: 0.3705, g_loss: 3.338
[38/51] d_loss: 0.5492, g_loss: 2.883
[38/51] d_loss: 0.5112, g_loss: 2.383
[38/51] d_loss: 0.8387, g_loss: 0.7617
[38/51] d_loss: 0.6059, g_loss: 3.063
[38/51] d_loss: 0.8333, g_loss: 2.041
[38/51] d_loss: 1.266, g_loss: 2.783
[38/51] d_loss: 1.666, g_loss: 2.288
[38/51] d_loss: 0.5533, g_loss: 1.281
[38/51] d_loss: 1.013, g_loss: 2.149
[38/51] d_loss: 0.5887, g_loss: 3.977
[38/51] d_loss: 0.

[40/51] d_loss: 0.7599, g_loss: 1.906
[40/51] d_loss: 0.7788, g_loss: 2.729
[40/51] d_loss: 0.3968, g_loss: 1.917
[40/51] d_loss: 1.021, g_loss: 1.664
[40/51] d_loss: 0.4408, g_loss: 1.362
[40/51] d_loss: 0.9044, g_loss: 1.263
[40/51] d_loss: 1.049, g_loss: 3.047
[40/51] d_loss: 0.7727, g_loss: 1.303
[40/51] d_loss: 1.424, g_loss: 0.8656
[40/51] d_loss: 0.7017, g_loss: 2.379
[40/51] d_loss: 1.06, g_loss: 2.72
[40/51] d_loss: 0.6938, g_loss: 1.209
[40/51] d_loss: 0.6881, g_loss: 2.155
[40/51] d_loss: 0.6513, g_loss: 1.929
[41/51] d_loss: 0.6784, g_loss: 2.685
[41/51] d_loss: 0.6881, g_loss: 0.9049
[41/51] d_loss: 0.9219, g_loss: 3.542
[41/51] d_loss: 0.6419, g_loss: 2.063
[41/51] d_loss: 0.4104, g_loss: 2.014
[41/51] d_loss: 0.4244, g_loss: 2.589
[41/51] d_loss: 0.5305, g_loss: 1.949
[41/51] d_loss: 0.9233, g_loss: 1.228
[41/51] d_loss: 0.5307, g_loss: 2.207
[41/51] d_loss: 1.595, g_loss: 2.354
[41/51] d_loss: 0.3733, g_loss: 2.603
[41/51] d_loss: 0.4559, g_loss: 3.089
[41/51] d_loss: 0

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D,GlobalAveragePooling2D,LeakyReLU,Conv2DTranspose,Activation,BatchNormalization
from keras.optimizers import Adam

from keras.layers import Dense
from keras import initializers
from keras.layers.core import Dropout



noise_dim = 1

def build_generator(input_shape):

    generator = Sequential()
    
    generator.add(Dense(256,
                        input_dim = noise_dim))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    
#     generator.add(Dense(256))
#     generator.add(BatchNormalization())
#     generator.add(LeakyReLU(0.2))
    

    


    
    generator.add(Dense(105, activation='tanh'))
    return generator


def build_discriminator(input_shape):

    discriminator = Sequential()
    discriminator.add(Dense(256,
                            input_dim=105))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    
#     discriminator.add(Dense(256))
#     discriminator.add(LeakyReLU(0.2))
#     discriminator.add(Dropout(0.3))
    

        

    
    discriminator.add(Dense(1, activation='sigmoid'))

    return discriminator

def build_functions(batch_size, noise_size, image_size, generator, discriminator):

    noise = K.random_normal((batch_size,) + noise_size,0.0,1.0,"float32")
    real_image = K.placeholder((batch_size,) + image_size)

    fake_image = generator(noise)

    d_input = K.concatenate([real_image, fake_image], axis=0)
    pred_real, pred_fake = tf.split(discriminator(d_input), num_or_size_splits = 2, axis = 0)

    pred_real = K.clip(pred_real,K.epsilon(),1-K.epsilon())
    pred_fake = K.clip(pred_fake,K.epsilon(),1-K.epsilon())

    d_loss = -(K.mean(K.log(pred_real)) + K.mean(K.log(1-pred_fake)))
    g_loss = -K.mean(K.log(pred_fake))

    # get updates of mean and variance in batch normalization layers
    d_updates = discriminator.get_updates_for([d_input])
    g_updates = generator.get_updates_for([noise])

    d_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(d_loss, discriminator.trainable_weights)
    d_train = K.function([real_image, K.learning_phase()], [d_loss],d_updates + d_training_updates)

    g_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(g_loss, generator.trainable_weights)
    g_train = K.function([real_image, K.learning_phase()], [g_loss], g_updates + g_training_updates)

    return d_train,g_train

In [ ]:
import keras

#from gan_libs.DCGAN import build_generator, build_discriminator, build_functions
# from gan_libs.LSGAN import build_generator, build_discriminator, build_functions
# from gan_libs.SNGAN import build_generator, build_discriminator, build_functions
# from gan_libs.WGAN_GP import build_generator, build_discriminator, build_functions

from utils.common import set_gpu_config, predict_images
from utils.draw_pose import draw_pose
import numpy as np

#set_gpu_config("0",0.5)

epoch = 50 + 1
image_size = (1,1,105)
noise_size = (1,1,1)
batch_size = 16

x_train = myData
np.random.shuffle(x_train)

generator = build_generator(noise_size)
print('gen params:',generator.count_params())
discriminator = build_discriminator(image_size)
print('dis params:',discriminator.count_params())
d_train, g_train = build_functions(batch_size, noise_size, image_size, generator, discriminator)

# generator.load_weights("e30_generator.h5".format(e))
# discriminator.load_weights("e30_discriminator.h5".format(e))


number_of_all_data = x_train.shape[0]
number_of_batches = int(number_of_all_data/batch_size)
print('Number of Batches passed in each epoch: ',number_of_batches)

import time
start_time = time.time()
is_first_epoch = True

for e in range(epoch):
    index = 0
    for batch in range(number_of_batches):
        real_images = x_train[index:index+batch_size]
        index =+ batch_size
        real_images.shape = (batch_size,1,1,105)
        d_loss, = d_train([real_images, 1])
        g_loss, = g_train([real_images, 1])
        if np.random.randint(low = 0, high = 100) == 1:   
            print ("[{0}/{1}] d_loss: {2:.4}, g_loss: {3:.4}".format(e, epoch, d_loss, g_loss))
            #generating a sample
            image = generator.predict(np.zeros(shape=(1,1)))
            image = np.array(image)
            draw_pose(image.reshape(105),'output',"_OneLatent_1_e{0}_batch{1}".format(e,batch))
    
    if(is_first_epoch):
        elapsed_time = time.time() - start_time 
        print('\n\nTime Taken for single epoch:')
        print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
        is_first_epoch = False
    
    if e % 50 == 0 and e > 0:
        generator.save_weights("_OneLatent_1_e{0}_generator.h5".format(e))
        discriminator.save_weights("_OneLatent_1_e{0}_discriminator.h5".format(e))
 
#just monitoring:
##########################################################################################
elapsed_time = time.time() - start_time 
print('\n\n\n\nNumber of parameter for the Generator and discriminator respectively:\n')
print(generator.count_params())
print('')
print(discriminator.count_params())
print('\n\nNumber of Epochs and steps for each epoch:\n')
print('epochs: ',epoch, '   batches: ', number_of_batches)

print('\n\nTime Taken:')
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
##########################################################################################
#NOTE: previously in each epoch 1000 steps were iterated. in each iteration data was permitated and a batch was chosen. 
#now data is shuffled first and there are number_of_data/batch_size steps to be passed in a single epoch. 
# since 200 epochs and 1000 steps each was used, now 25 epochs and ˜7300 steps is used so the balance is kept.

In [ ]:
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D,GlobalAveragePooling2D,LeakyReLU,Conv2DTranspose,Activation,BatchNormalization
from keras.optimizers import Adam

from keras.layers import Dense
from keras import initializers
from keras.layers.core import Dropout



noise_dim = 1

def build_generator(input_shape):

    generator = Sequential()
    
    generator.add(Dense(256,
                        input_dim = noise_dim))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    
#     generator.add(Dense(256))
#     generator.add(BatchNormalization())
#     generator.add(LeakyReLU(0.2))
    

    


    
    generator.add(Dense(105, activation='tanh'))
    return generator


def build_discriminator(input_shape):

    discriminator = Sequential()
    discriminator.add(Dense(256,
                            input_dim=105))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    
#     discriminator.add(Dense(256))
#     discriminator.add(LeakyReLU(0.2))
#     discriminator.add(Dropout(0.3))
    

        

    
    discriminator.add(Dense(1, activation='sigmoid'))

    return discriminator

def build_functions(batch_size, noise_size, image_size, generator, discriminator):

    noise = K.random_normal((batch_size,) + noise_size,0.0,3.0,"float32")
    real_image = K.placeholder((batch_size,) + image_size)

    fake_image = generator(noise)

    d_input = K.concatenate([real_image, fake_image], axis=0)
    pred_real, pred_fake = tf.split(discriminator(d_input), num_or_size_splits = 2, axis = 0)

    pred_real = K.clip(pred_real,K.epsilon(),1-K.epsilon())
    pred_fake = K.clip(pred_fake,K.epsilon(),1-K.epsilon())

    d_loss = -(K.mean(K.log(pred_real)) + K.mean(K.log(1-pred_fake)))
    g_loss = -K.mean(K.log(pred_fake))

    # get updates of mean and variance in batch normalization layers
    d_updates = discriminator.get_updates_for([d_input])
    g_updates = generator.get_updates_for([noise])

    d_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(d_loss, discriminator.trainable_weights)
    d_train = K.function([real_image, K.learning_phase()], [d_loss],d_updates + d_training_updates)

    g_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(g_loss, generator.trainable_weights)
    g_train = K.function([real_image, K.learning_phase()], [g_loss], g_updates + g_training_updates)

    return d_train,g_train

In [ ]:
import keras

#from gan_libs.DCGAN import build_generator, build_discriminator, build_functions
# from gan_libs.LSGAN import build_generator, build_discriminator, build_functions
# from gan_libs.SNGAN import build_generator, build_discriminator, build_functions
# from gan_libs.WGAN_GP import build_generator, build_discriminator, build_functions

from utils.common import set_gpu_config, predict_images
from utils.draw_pose import draw_pose
import numpy as np

#set_gpu_config("0",0.5)

epoch = 50 + 1
image_size = (1,1,105)
noise_size = (1,1,1)
batch_size = 16

x_train = myData
np.random.shuffle(x_train)

generator = build_generator(noise_size)
print('gen params:',generator.count_params())
discriminator = build_discriminator(image_size)
print('dis params:',discriminator.count_params())
d_train, g_train = build_functions(batch_size, noise_size, image_size, generator, discriminator)

# generator.load_weights("e30_generator.h5".format(e))
# discriminator.load_weights("e30_discriminator.h5".format(e))


number_of_all_data = x_train.shape[0]
number_of_batches = int(number_of_all_data/batch_size)
print('Number of Batches passed in each epoch: ',number_of_batches)

import time
start_time = time.time()
is_first_epoch = True

for e in range(epoch):
    index = 0
    for batch in range(number_of_batches):
        real_images = x_train[index:index+batch_size]
        index =+ batch_size
        real_images.shape = (batch_size,1,1,105)
        d_loss, = d_train([real_images, 1])
        g_loss, = g_train([real_images, 1])
        if np.random.randint(low = 0, high = 100) == 1:   
            print ("[{0}/{1}] d_loss: {2:.4}, g_loss: {3:.4}".format(e, epoch, d_loss, g_loss))
            #generating a sample
            image = generator.predict(np.zeros(shape=(1,1)))
            image = np.array(image)
            draw_pose(image.reshape(105),'output',"_OneLatent_3_e{0}_batch{1}".format(e,batch))
    
    if(is_first_epoch):
        elapsed_time = time.time() - start_time 
        print('\n\nTime Taken for single epoch:')
        print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
        is_first_epoch = False
    
    if e % 50 == 0 and e > 0:
        generator.save_weights("_OneLatent_3_e{0}_generator.h5".format(e))
        discriminator.save_weights("_OneLatent_3_e{0}_discriminator.h5".format(e))
 
#just monitoring:
##########################################################################################
elapsed_time = time.time() - start_time 
print('\n\n\n\nNumber of parameter for the Generator and discriminator respectively:\n')
print(generator.count_params())
print('')
print(discriminator.count_params())
print('\n\nNumber of Epochs and steps for each epoch:\n')
print('epochs: ',epoch, '   batches: ', number_of_batches)

print('\n\nTime Taken:')
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
##########################################################################################
#NOTE: previously in each epoch 1000 steps were iterated. in each iteration data was permitated and a batch was chosen. 
#now data is shuffled first and there are number_of_data/batch_size steps to be passed in a single epoch. 
# since 200 epochs and 1000 steps each was used, now 25 epochs and ˜7300 steps is used so the balance is kept.

In [ ]:
    #gpu details:
    ################----------------------------
from six.moves import cStringIO as StringIO
import gpustat

gpustats = gpustat.new_query()
fp = StringIO()
gpustats.print_formatted(
     fp=fp, no_color=False, show_user=False,
     show_cmd=False, show_pid=False, show_power=False, show_fan_speed=False)

result = fp.getvalue()
print('\n\n')
print(result)
    ################----------------------------

In [ ]:
generator.load_weights("e400_generator.h5".format(e))
discriminator.load_weights("e400_discriminator.h5".format(e))

In [ ]:
z = -1;
for i in np.linspace(-3,3,100):
    z+=1
    x= np.array((-i,0,0,i,0)).reshape(1,5)
    image = generator.predict(x)
    image = np.array(image)
    draw_pose(image.reshape(105),'output',"e{0}".format(z))